# Datenanalyse


---

Datensatz: [Synthea Breast Cancer Dataset](https://github.com/Fuenfgeld/DMA2023TeamA/tree/main/Daten/Quelldaten)

Primär- und Fremdschlüsseldefinitionen: [Synthea GitHub Repository](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary)

Projektgruppe GitHub Repository: [DMA2023TeamA](https://github.com/Fuenfgeld/DMA2023TeamA)

Source-DB: [GoogleDrive Ablage](https://drive.google.com/drive/folders/1k5cfjGXjNHmwQkydzjTdVHoBvCniBU_W), erstellt mit [Setup_and_fill_Database.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/Setup_and_fill_Database.ipynb)

Data Warehouse-Datenbank: [GoogleDrive Ablage](https://drive.google.com/file/d/1l-HcqCezubHnR737DkbiRzdHanP7_g_D), erstellt mit [ETL_process.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/ETL_Process.ipynb)


*Version*: 0.1 

Version Date: 05/02/2023

Changes: 

*   



In [ ]:
# Vorsichtshalber: Löschen aller Variablen
%reset -f

# Laden der benötigten Libraries
from google.colab import drive
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd

In [ ]:
# Google Drive mounten, force_remount auf True setzen, damit ein Remount erzwungen wird
drive.mount('/content/gdrive/', force_remount=True)

# Datenbankordner auf dem Shareddrive checken, es müssen source_breast_cancer.db und DWH_breast_cancer.db vorhanden sein
!ls "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank"

# Patiententyp festlegen
patient_type = "breast_cancer"

# Pfad zur DWH-Datenbank setzen
DB_DWH_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank/DWH_breast_cancer.db"

# Check
print("\n" + DB_DWH_PATH)

# Versionen der verwendeten Pakete abfragen 

Die Versionen der verwendeten Python-Installation und der Python-Pakete abfragen. 

In [ ]:
# Python-Version
print("Python-Version:")
!python --version


# Pandas-Version
print("\n" + "Pandas-Version:")
print("Pandas " + pd.__version__)

# sqlitee-Version
print("\n" + "sqlite3-Version:")
print("sqlite3 " + sq.sqlite_version)


# Extraktion der Daten aus der DWH-Datenbank


In [ ]:
# Datenbankverbindung zum DWH aufbauen
dwh_conn = sq.connect(DB_DWH_PATH) 
if dwh_conn is not None:
  dwh_cursor = dwh_conn.cursor()
else:
  print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Tabellennamen aus der Datenbank ziehen
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = dwh_cursor.fetchall()
tablelist

In [ ]:
# SQL Abfrage zur Ausleitung der Daten zu den Behandlungskosten
extract_costs = """SELECT enc.id,
                enc.patient,
                pat.gender    AS PAT_GENDER,
                pat.race      AS PAT_RACE,
                enc.enc_payer,
                pay.NAME      AS ENC_PAYER_NAME,
                enc.enc_base_cost,
                enc.enc_total_claim_cost,
                enc.enc_payer_coverage,
                enc.pro_base_cost,
                enc.med_base_cost,
                enc.med_dispenses,
                enc.med_total_cost,
                enc.med_payer_coverage,
                enc.med_payer,
                enc.con_code,
                sct.term      AS CON_TERM
                FROM  f_encounter_costs AS enc
                      JOIN d_patients AS pat
                        ON enc.patient = pat.id
                      JOIN d_payers AS pay
                        ON enc.enc_payer = pay.id
                      LEFT JOIN d_snomedct AS sct
                        ON enc.con_code = sct.code
                ORDER  BY patient, enc.id;"""
data_costs = pd.read_sql(extract_costs, dwh_conn)
print(data_costs.head())  

In [ ]:
# SQL Abfrage zur Ausleitung der Diagnosedaten
extract_conditions = """SELECT enc.patient,
                               enc.con_start,
                               enc.con_code,
                               sct.term
                        FROM   f_encounter_costs AS enc
                               JOIN d_snomedct AS sct
                                 ON enc.con_code = sct.code
                        ORDER  BY enc.patient,
                                  enc.con_code;"""
data_conditions = pd.read_sql(extract_conditions, dwh_conn)
print(data_conditions.head())  

In [ ]:
# SQL Abfrage zur Ausleitung der Diagnoseart (none = keine, breast_cancer = Brustkrebs, other = andere Diagnose)
extract_diagnoses = """SELECT pat.id AS patient,
                              CASE
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  enc.con_code = "254837009") THEN 'breast_cancer'
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  NOT enc.con_code = "254837009"
                                                        AND enc.con_code IS NOT NULL) THEN 'other'
                                ELSE 'none'
                              END    AS diagnosis
                        FROM   d_patients AS pat;"""
data_diagnoses = pd.read_sql(extract_diagnoses, dwh_conn)
print(data_diagnoses.head())  

In [ ]:
# Commit und Close
conn.commit()
conn.close()

# EDA